In [1]:
from torchvision import datasets, transforms
import torch

transform = transforms.Compose([
     transforms.ToTensor()
 ])

train_data = datasets.FashionMNIST(root='data', train=True, download=True , transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 17.6MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 338kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.40MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 1.27MB/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        out1 = torch.relu(self.fc1(x))
        out2 = self.fc2(out1)
        return out1, out2


In [9]:
MLP_model = MLP()
MLP_model.load_state_dict(torch.load('mlp_model_weights.pth'))
state_dict = torch.load('mlp_model_weights.pth')


<ipython-input-9-8eea81d9d999>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  MLP_model.load_state_dict(torch.load('mlp_model_weights.pth'))
<ipython-input-9-8eea81d9d999>

In [10]:
def relu(x):
    return torch.max(torch.zeros_like(x), x)


In [13]:
def MyModel(x):
    x = x.view(-1, 28 * 28)
    out1 = relu(x @ state_dict['fc1.weight'].T + state_dict['fc1.bias'])
    out2 = out1 @ state_dict['fc2.weight'].T + state_dict['fc2.bias']
    return out1 , out2


In [16]:

image, label = test_data[0]
image = image.unsqueeze(0)

out1_main, out2_main = MLP_model(image)


out1_custom ,out2_custom = MyModel(image)


print("Hidden Layer in torch:", torch.sum(torch.abs(out1_main)).item())
print("Hidden Layer in mymodel:", torch.sum(torch.abs(out1_custom)).item())
print("Output Layer in torch:", torch.sum(torch.abs(out2_main)).item())
print("Output Layer in mymodel:", torch.sum(torch.abs(out2_custom)).item())


Hidden Layer in torch: 106.80531311035156
Hidden Layer in mymodel: 106.80531311035156
Output Layer in torch: 259.7275085449219
Output Layer in mymodel: 259.7275085449219


Step 5